In [5]:
from ase.io import read
from ase.visualize import view
crystals = ['TiN', 'VN', 'ScN', 'ZrN', 'NbN']  # Assuming these are defined elsewhere

for crystal_name in crystals:
    filename = f'{crystal_name}_PBE+D3_strain.traj'
    crystal = read(filename)
    print(crystal.cell.cellpar())
    view(crystal)  # This will open a window to visualize the crystal structure
    print(f"Visualizing {crystal_name} structure from {filename}")

[ 2.49867018  2.49867018  2.49867018 90.         90.         90.        ]
Visualizing TiN structure from TiN_PBE+D3_strain.traj
[ 2.45212166  2.45212166  2.45212166 90.         90.         90.        ]
Visualizing VN structure from VN_PBE+D3_strain.traj
[ 2.45981341  2.45981341  2.45981341 90.         90.         90.        ]
Visualizing ScN structure from ScN_PBE+D3_strain.traj
[ 2.57823395  2.57823395  2.57823395 90.         90.         90.        ]
Visualizing ZrN structure from ZrN_PBE+D3_strain.traj
[ 2.55999354  2.55999354  2.55999354 90.         90.         90.        ]
Visualizing NbN structure from NbN_PBE+D3_strain.traj


In [ ]:
from ase.io import read, write
from ase.atoms import Atoms
import os

materials = ['TiN', 'VN', 'ScN', 'NbN', 'ZrN']

for material in materials:
    print(f"Processing {material} to fix atom indexing...")
    

    crystal = read(material + '.cif')  # Read the crystal structure from CIF file

    # Get a list of atoms with their chemical symbols and original indices
    # We will sort this list to create a new Atoms object with desired order
    atom_list = []
    for atom in crystal:
        atom_list.append((atom.symbol, atom.position, atom.tag, atom.momentum, atom.mass, atom.magmom))
    
    # Sort atoms first by chemical symbol (e.g., 'N' before 'Nb', 'Ti' before 'N')
    # This assumes consistent naming for metal and nitrogen atoms.
    # For example, if 'M' is the metal and 'N' is nitrogen, 'M' will come before 'N' due to alphabetical sort.
    atom_list.sort(key=lambda x: x[0]) 

    # Create a new Atoms object with the reordered atoms
    # Extract positions, symbols, and cell/pbc from original crystal
    new_symbols = [item[0] for item in atom_list]
    new_positions = [item[1] for item in atom_list]
    
    reordered_crystal = Atoms(symbols=new_symbols,
                              positions=new_positions,
                              cell=crystal.get_cell(),
                              pbc=crystal.get_pbc())

    # Add tags, momentum, mass, magmom if they were present
    if crystal.has('tags'):
        reordered_crystal.set_tags([item[2] for item in atom_list])
    if crystal.has('momenta'):
        reordered_crystal.set_momenta([item[3] for item in atom_list])
    if crystal.has('masses'):
        reordered_crystal.set_masses([item[4] for item in atom_list])
    if crystal.has('magmoms'):
        reordered_crystal.set_magmoms([item[5] for item in atom_list])

    # Save the reordered crystal to an .xyz file
    xyz_filename = os.path.join(fixed_index_dir, f'{material}_reordered.xyz')
    write(xyz_filename, reordered_crystal)
    
    print(f"Reordered atoms and saved to {xyz_filename}")
    print(f"New atom order for {material}: {reordered_crystal.get_chemical_formula()} (original number of atoms: {len(crystal)})")

print("\nAtom indexing fix process completed for all materials.")